# Part 4: Using GPU acceleration with PyTorch

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 3.5MB/s eta 0:00:01


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [ ]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 0.1663, -1.1947,  0.4580,  ...,  2.3011, -0.1942,  0.3566],
        [-0.7079,  0.0406, -0.8608,  ...,  0.5107, -0.5505,  0.4528],
        [ 0.6633, -0.1831, -0.0461,  ..., -0.0226, -0.0792,  2.1576],
        ...,
        [ 0.7681, -0.2165, -1.6709,  ...,  1.4140,  1.1224, -0.5862],
        [ 0.4910,  0.0487,  0.0463,  ...,  0.5802, -0.8737,  0.1851],
        [-0.2770, -0.2192, -0.3325,  ..., -2.2176, -0.3318, -0.8116]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [ ]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-0.2061, -0.8590,  0.0114,  ..., -0.9519, -0.7694,  0.7434],
        [-1.8661, -0.0572,  1.0479,  ...,  0.2048, -0.7446,  0.1454],
        [-0.0817,  0.3811, -1.0022,  ..., -0.9220, -0.3981, -0.3742],
        ...,
        [-0.0352, -0.0200, -0.0361,  ...,  0.0921, -0.0106, -0.1625],
        [ 1.0550,  0.0381,  0.2507,  ...,  0.3160,  1.2748, -0.0150],
        [-0.2233,  0.1195,  1.3133,  ..., -0.0041,  0.0719,  0.0591]],
       device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[ 0.8519, -0.9106,  0.1294,  ...,  0.5355,  0.0397, -1.0819],
        [-0.6490, -1.2124,  1.2996,  ...,  0.1148, -0.2391,  3.2560],
        [ 0.0494, -0.6550, -0.5106,  ...,  0.0420,  0.0935, -2.1921],
        ...,
        [ 0.0621, -0.1362,  0.0183,  ...,  2.2961, -0.2443, -0.8645],
        [ 0.0512, -0.6100, -0.7912,  ...,  0.4100, -0.0131,  1.0929],
        [ 1.0651, -1.1793, -2.0469,  ...,  2.4314,  1.1930, -0.2294]],
       device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

CPUs are designed for more general computing workloads.  GPUs in contrast are less flexible, however GPUs are designed to compute in parallel the same instructions.   Deep Neural Networks (DNN) are structured in a very uniform manner such that at each layer of the network thousands of identical artificial neurons perform the same computation.   Therefore the structure of a DNN fits quite well with the kinds of computation that a GPU can efficiently perform.

GPUs have additional advantages over CPUs,  these include having more computational units and having a higher bandwidth to retrieve from memory.  Furthermore,  in applications requiring image processing (i.e. Convolution Neural Networks) GPU graphics specific capabilities can be exploited to further speed up calculations.  

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

The primary weakness of GPUs as compared to CPUs is memory capacity on GPUs are lower than CPUs.   The highest known GPU contains 24GB of RAM, in contrast, CPUs can reach 1TB of RAM.   A secondary weakness is that a CPU is required to transfer data into the GPU card.  